# 4 - Creating New User Profiles Using the SEEK WriteAPI

In [1]:
import requests
import json
import string
from pprint import pprint

The base_url now points to a special SEEK instance that we can do what we like with.

In [2]:
base_url = 'http://0.0.0.0:3000' # here you can use any SEEK instances

In addition to the standard headers for sending a "GET" request, we include the API token in the header for authentication.

The token acts as a key to access the API, allowing the server to verify the client's identity and permissions.

The value "Token {your_token_created_in_your_SEEK_instance}" is a placeholder for a real API token. 

In a production setup, you should:
 - Replace it with your actual token.
 - Store the token securely (e.g., in environment variables or a secrets manager).

In [ ]:
headers = {
    "Accept": "application/json",               # Specifies the client expects the response in JSON format.
    "Content-Type": "application/json",         # Informs the server that the request body is in JSON format.
    # "Authorization": "Token {your_token_created_in_your_SEEK_instance}"  # Provides the API token for authentication.
}

The init_json_data function is designed to prepare JSON data that can be used as the body content in an API POST request. 

In [4]:
def init_json_data(file, title=None):
    json_format = json.load(open(file))
    if (title is not None):
        json_format['data']['attributes']['title'] = title
    json_data = json.dumps(json_format)
    return json_data

### Post a person profile

In [5]:
# Define the URL for the "people" resource endpoint by appending to the base URL
url = base_url + "/people"

# Specify the path to the local JSON file containing the data for the person to be created
person_file = "./json/person_example.json"

# Initialize the JSON data by loading it from the specified file
json_data = init_json_data(person_file)

json_data

'{"data": {"type": "people", "attributes": {"first_name": "Sophia", "last_name": "Schneider", "email": "s.schneiderhu@gmail.org", "description": "Research Associate", "web_page": "https://www.h-its.org", "phone": "+49 (0)0000000000", "skype_name": "s.schneider", "expertise": ["GOT", "programming"], "tools": ["javascript", "C++", "Python", "Ruby"]}, "relationships": {"projects": {"data": [{"id": "2", "type": "projects"}]}}}}'

In [6]:
# Send a POST request to the API to create a new person resource
# Include the necessary headers for authentication and content type, and the JSON data as the request body
response = requests.post(url, headers=headers, data=json_data)

# Parse the response as JSON
result = response.json()

# Print out the result 
print(base_url + '/people/' + str(result['data']['id']))
print(result)

http://0.0.0.0:3000/people/280
{'data': {'id': '280', 'type': 'people', 'attributes': {'avatar': None, 'title': 'Sophia Schneider', 'description': 'Research Associate', 'first_name': 'Sophia', 'last_name': 'Schneider', 'orcid': None, 'mbox_sha1sum': 'a8eec824b8708e44b8bd3406351b7696c9dad422', 'expertise': ['GOT', 'programming'], 'tools': ['C++', 'Python', 'Ruby', 'javascript'], 'login': None}, 'relationships': {'projects': {'data': []}, 'institutions': {'data': []}, 'investigations': {'data': []}, 'studies': {'data': []}, 'assays': {'data': []}, 'data_files': {'data': []}, 'models': {'data': []}, 'sops': {'data': []}, 'publications': {'data': []}, 'presentations': {'data': []}, 'events': {'data': []}, 'documents': {'data': []}, 'workflows': {'data': []}, 'collections': {'data': []}}, 'links': {'self': '/people/280'}, 'meta': {'created': '2024-12-02T14:19:23.000Z', 'modified': '2024-12-02T14:19:23.000Z', 'api_version': '0.3', 'base_url': 'http://localhost:3000', 'uuid': '5a0f3120-92e6-0

### Update a person profile

In [7]:
# Step 1: Read the JSON from the file
person_file = "./json/update_person_example.json"

with open(person_file, "r") as file:
    json_data = json.load(file)  # Parse the JSON file

# Step 2: Replace "THE_ID" with the actual ID from the response
real_id = str(result['data']['id'])  # Assuming result is already obtained from a previous API response

json_data_str = json.dumps(json_data)  # Convert dict to string to perform replace


# Replace the placeholder "THE_ID" in the string
json_data_str = json_data_str.replace(json_data['data']['id'], str(result['data']['id']))


# Convert the string back to a JSON object
json_data_updated = json.loads(json_data_str)


# Step 3: Send the updated JSON data via a PUT request to update the person
person_update_url = base_url + '/people/' + real_id  # Construct the URL for the specific person resource

response = requests.put(person_update_url, headers=headers, json=json_data_updated)

# Check and print the response
if response.status_code == 200:
    print("Person updated successfully:", response.json())
else:
    print(f"Failed to update the person. Status code: {response.status_code}")
    print("Response:", response.text)


Person updated successfully: {'data': {'id': '280', 'type': 'people', 'attributes': {'avatar': None, 'title': 'Patched Name', 'description': 'A person with all possible details', 'first_name': 'Patched', 'last_name': 'Name', 'orcid': 'https://orcid.org/0000-0001-9842-9718', 'mbox_sha1sum': '5f9cd9691c643af7e4506ff02cdb6629b9bb880f', 'expertise': ['modeling', 'programming'], 'tools': ['CeriusII', 'Gromacs', 'Python'], 'login': None}, 'relationships': {'projects': {'data': []}, 'institutions': {'data': []}, 'investigations': {'data': []}, 'studies': {'data': []}, 'assays': {'data': []}, 'data_files': {'data': []}, 'models': {'data': []}, 'sops': {'data': []}, 'publications': {'data': []}, 'presentations': {'data': []}, 'events': {'data': []}, 'documents': {'data': []}, 'workflows': {'data': []}, 'collections': {'data': []}}, 'links': {'self': '/people/280'}, 'meta': {'created': '2024-12-02T14:19:23.000Z', 'modified': '2024-12-02T14:19:23.000Z', 'api_version': '0.3', 'base_url': 'http://l

### Create a list of profiles 

The init_json_data_replacement function is designed to load JSON data from a file and dynamically replace specific placeholders with their corresponding values, allowing for flexible and reusable JSON payloads for API requests.

In [8]:
REPLACEMENTS = {
    'FIRST_NAME': "first_name",
    'LAST_NAME': "last_name", 
    'DESCRIPTION': "web_page",
    'WEBSITE': "1",
    'ORCID':"orcid",
    'PHONE_NUMBER':"phone"
}

In [9]:
def init_json_data_replacement(file, replacements=REPLACEMENTS):
    json_format = json.load(open(file))
    json_data = json.dumps(json_format)
    for key, value in replacements.items():
        json_data = str.replace(json_data, key, value)
    return json_data

1. Read the Excel File:
We use pandas to read the Excel file.
2. Loop Through Each Person:
For each row in the file, we update a template with the person's details, such as their first name, last name, description, etc.
3. Prepare JSON Data:
We use the init_json_data_replacement() function to create a JSON object with the person's details, based on a template file. 
4.  Send Data to SEEK:
We use requests.post() to send the data to SEEK’s API and create a new person resource.
5. Print the Result:
After sending the data, we print the URL of the newly created person and some of their details from the response.

In [10]:
import pandas as pd

peoplelist_file = 'peoplelist.xlsx'
peoplelist = pd.read_excel(peoplelist_file,dtype=str)
peoplelist.fillna('', inplace=True)

for index, row in peoplelist.iterrows():

    REPLACEMENTS['FIRST_NAME'] = row['first_name']
    REPLACEMENTS['LAST_NAME'] = row['last_name']
    REPLACEMENTS['DESCRIPTION'] = row['description']
    REPLACEMENTS['WEBSITE'] = row['web_page']
    REPLACEMENTS['ORCID'] = row['orcid']
    REPLACEMENTS['PHONE_NUMBER'] = str(row['phone'])
    
    person_file = "./json/person_template.json"

    json_data = init_json_data_replacement(person_file, REPLACEMENTS)
    
    
    response = requests.post(url, headers=headers, data=json_data)

    result = response.json()
    print(base_url + '/people/' + str(result['data']['id']))
    print(result['data']['attributes'])
    
    
    print("------------------------------------------")

http://0.0.0.0:3000/people/281
{'avatar': None, 'title': 'first1 last1', 'description': 'description of first1.last1', 'first_name': 'first1', 'last_name': 'last1', 'orcid': 'https://orcid.org/0000-0002-8318-3222', 'mbox_sha1sum': 'ec53e28650b30ad0949cf97370f57d1ea17dc4d3', 'expertise': [], 'tools': [], 'login': None}
------------------------------------------
http://0.0.0.0:3000/people/282
{'avatar': None, 'title': 'first2 last2', 'description': 'description of first2.last2', 'first_name': 'first2', 'last_name': 'last2', 'orcid': None, 'mbox_sha1sum': '4452d7bc1ec949101458d2b856732f177a970dde', 'expertise': [], 'tools': [], 'login': None}
------------------------------------------
http://0.0.0.0:3000/people/283
{'avatar': None, 'title': 'first3 last3', 'description': 'description of first3.last3', 'first_name': 'first3', 'last_name': 'last3', 'orcid': None, 'mbox_sha1sum': '7c12cffea3bebabfa611fc403a105b92e5adbe11', 'expertise': [], 'tools': [], 'login': None}
------------------------